# Récupération et traitement des données

In [119]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import pynsee as yns
import requests
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from cartiflette.s3 import download_vectorfile_url_all

In [120]:
from pathlib import Path

In [121]:
Path.cwd()

PosixPath('/home/cathu/Documents/ENSAE/Projet_Catherine_Christelle')

## Les aménagements cyclables en Ile de France

In [122]:
a_velo= gpd.read_file('amenagements-velo-en-ile-de-france.geojson')

In [5]:
# Nombre d'entrées
a_velo.shape[0]

112217

In [6]:
a_velo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [7]:
a_velo['longueur'].describe()

count    112217.000000
mean        126.315959
std         174.519817
min           0.000000
25%          24.000000
50%          70.000000
75%         161.000000
max        3984.000000
Name: longueur, dtype: float64

In [8]:
a_velo.sample(n=1)

,osm_id,nom_com,sens_voit,ag,panneaux,moyenn_ech,revetement,highway,insee_com,nom_voie,longueur,petite_ech,nv,ad,geometry
100698,792297373.0,Mézières-sur-Seine,UNIQUE,None,None,22,asphalt,secondary,78402,Route de Paris à Cherbourg,23,2,None,bande uni,"LINESTRING (1.79807 48.96214, 1.79776 48.96217)"


Pour le moment, nous travaillons sans les données de géolocalisation qui ne sont pas nécessaires aux calculs. Nous créons pour cela une nouvelle dataframe

In [9]:
col_to_keep = ["nom_com", "sens_voit", "ag", "panneaux", "revetement", "highway", "insee_com", "longueur", "nv", "ad"] 

In [52]:
# df_a_velo = a_velo[col_to_keep].copy()

df_a_velo = a_velo

print(df_a_velo.head())

         osm_id            nom_com sens_voit                              ag  \
0  4.014620e+08            Chelles    DOUBLE                            None   
1  4.048666e+08   La Queue-en-Brie        NC  chemin service site propre uni   
2  1.154304e+09            Lésigny        NC                  voie verte uni   
3  3.300503e+08  Pontault-Combault    UNIQUE                       DSC bande   
4  1.104033e+09   Champs-sur-Marne    UNIQUE                            None   

  panneaux moyenn_ech revetement      highway insee_com  \
0     None         32       None      service     77108   
1     None         11       None        track     94060   
2     None         11    asphalt         path     77249   
3     None         22       None  residential     77373   
4     None         22    asphalt  residential     77083   

                     nom_voie  longueur petite_ech            nv  \
0  Rue de la Mare Longue Noue        24          3     limite 30   
1             Allée Jacquett

### Toutes les pistes ne se valent pas : création de variables d'étude pour les aménagements cyclables

Nous souhaitons étudier la répartition géograhique des aménagements cyclables et particulièrement leur densité en mètre par habitants. Toutefois, l'ensemble des pistes cyclables n'est pas de la même "qualité" : séparée de la route ou non, sens inverse de la circulation, type de revêtement, etc. Pour prendre en compte la qualité des aménagements cyclables, nous pouvons donner des coefficients aux mètres de pistes selon ces différents critères lors du calcul du nombre total de mètres aménagés par commune. Nous allons proposer plusieurs méthodes de calcul selon ces critères.

In [80]:
print(df_a_velo["sens_voit"].unique())
print(df_a_velo["ag"].unique())
print(df_a_velo["revetement"].unique())
print(df_a_velo["highway"].unique())
print(df_a_velo["nv"].unique())
print(df_a_velo["ad"].unique())

['DOUBLE' 'NC' 'UNIQUE']
[None 'chemin service site propre uni' 'voie verte uni' 'DSC bande'
 'autre chemin velo uni' 'piste uni' 'DSC' 'piste trottoir uni'
 'bande uni' 'cheminement trottoir uni' 'goulotte' 'chemin dedie uni'
 'chaucidou' 'cheminement uni' 'DSC piste' 'voie bus uni' 'piste bi'
 'bande bi' 'shoulder uni']
[None 'asphalt' 'compacted' 'concrete' 'unpaved' 'wood' 'paving_stones'
 'ground' 'sett' 'gravel' 'sand' 'paved' 'fine_gravel' 'dirt'
 'cobblestone' 'concrete:plates' 'concrete:lanes' 'grass'
 'cobblestone:flattened' 'metal' 'pebblestone' 'unhewn_cobblestone'
 'bricks' 'tartan' 'pavés' 'vegecol' 'artificial_turf' 'grass_paver'
 'earth' 'bitume' 'à_définir' 'rock']
['service' 'track' 'path' 'residential' 'footway' 'cycleway' 'secondary'
 'tertiary' 'living_street' 'unclassified' 'pedestrian' 'steps' 'primary'
 'primary_link' 'secondary_link' 'tertiary_link' 'trunk_link'
 'motorway_link']
['limite 30' 'hors voirie' 'unclassified' 'z20' 'rue pietonne' 'z30'
 'escalier ve

In [12]:
df_a_velo['sens_voit'].isna().sum()

0

In [13]:
df_a_velo['ag'].isna().sum()

68312

In [14]:
df_a_velo['revetement'].isna().sum()

38711

In [15]:
df_a_velo['highway'].isna().sum()

0

In [79]:
df_a_velo['nv'].isna().sum()

0

In [17]:
df_a_velo['ad'].isna().sum()

68522

In [75]:
print(df_a_velo['highway'].value_counts().get('unclassified', 0))

9548
4025


In [77]:
df_a_velo['adg'] = df_a_velo.apply(lambda row: row['ad'] if pd.notna(row['ad']) else (row['ag'] if pd.notna(row['ag']) else 'unclassified'), axis=1)

In [78]:
print(df_a_velo['adg'].value_counts().get('unclassified', 0))

55496


Seul le critère suivant ne présente pas trop de valeurs manquantes (nommées 'unclassified' dans la base) :  type de route ("highway", cf [documentation OpenstreeMap](https://wiki.openstreetmap.org/wiki/Key:highway) ). Les autres variables ont soit trop de valeurs manquantes soit manquent d'intérêt seule (sens des voitures). Cependant, "highway" est une colonne générale de catégorisation pour les données d'OpenStreetMap et manque de spécificité pour l'étude des pistes cyclables. Nous allons donc proposer deux méthodes de pondération (en plus d'une variable non pondérée) : une avec "highway", une avec "adg" (['nature de la voie'](https://opendata.stif.info/api/datasets/1.0/amenagements-velo-en-ile-de-france/attachments/metadonnees_amenagements_velo_en_ile_de_france_pdf/)), en proposant de normer les chemins dont nous n'avons pas connaissance de la qualité. Le nombre de valeurs manquantes est cependant très important, malgré notre tentative de réduire son ampleur en sommant ad et ag (ce sont les types de voies à gauche et à droite). Roulant à droite en, nous avons donner la priorité à voie à droite.

Nous allons produire 3 variables : deux pondérées censées prendre en compte la qualité de la route, et une autre sans pondération, pour la longueur d'aménagements cyclables par ville. Pour prendre en compte la qualité nous trions les types de routes en leur affectant des poids selon leurs caractéristiques : bande le long d'une route ou séparation, chemin avec ou sans voiture, avec ou sans piéton, etc. Pour cela, la variable "adg" nous semble plus adaptée car plus précise, mais elle a le défaut de présenter énormément valeurs manquantes. Lorsque la valeur est manquante nous assignons un poids neutre (1). La comparaison des résultats avec les 2 méthodes de pondération ou la méthode sans pondération pourra aussi nous renseigner sur la qualité de notre catégorisation.

In [81]:
print(df_a_velo["adg"].unique())

['unclassified' 'chemin service site propre uni' 'voie verte uni'
 'DSC bande' 'bande uni' 'autre chemin velo uni' 'piste uni' 'DSC'
 'piste trottoir uni' 'cheminement trottoir uni' 'goulotte'
 'chemin dedie uni' 'chaucidou' 'cheminement uni' 'voie bus uni'
 'DSC piste' 'piste bi' 'bande bi' 'shoulder uni']


In [99]:
highway_quality_mapping = {
    'service': 1,
    'track': 1,
    'path': 1,
    'trunk_link': 1,
    'motorway_link': 1,
    'residential': 2,
    'footway': 2,
    'cycleway': 2,
    'primary': 2,    
    'primary_link': 2,
    'unclassified': 2,
    'secondary': 3,
    'tertiary': 3,
    'secondary_link': 3,
    'tertiary_link': 3,
    'living_street': 3,
    'pedestrian': 4,
    'steps': 4
}


adg_quality_mapping = {
    'chemin service site propre uni': 1,
    'chemin dedie uni': 1,
    'voie verte uni': 1,
    'autre chemin velo uni': 1,
    'piste bi' : 1,
    'piste uni': 1,
    'bande uni': 2,
    'bande bi' : 2,
    'unclassified': 2,
    'cheminement trottoir uni' : 3,
    'piste trottoir uni': 3,
    'chaucidou': 3,
    'cheminement uni': 3,
    'shoulder uni' : 3,
    'voie bus uni': 4,
    'goulotte': 4,
    'DSC': 4,
    'DSC bande' : 4,
    'DSC piste' : 4
}

In [100]:
df_a_velo['qual_hw'] = df_a_velo['highway'].map(highway_quality_mapping)

df_a_velo['qual_adg'] = df_a_velo['adg'].map(adg_quality_mapping)

In [101]:
quality_weights = {
    1: 1.25,
    2: 1.,
    3: 0.75,
    4: 0.5
}

In [102]:
# Nouvelle colonne pour la longueur pondérée highway

df_a_velo['longueur_pond_hw'] = df_a_velo['longueur'] * df_a_velo['qual_hw'].map(quality_weights)

# Nouvelle colonne pour la longueur pondérée nature voie

df_a_velo['longueur_pond_adg'] = df_a_velo['longueur'] * df_a_velo['qual_adg'].map(quality_weights)

In [105]:
total_longueur_pond_hw = df_a_velo['longueur_pond_hw'].sum()
print(total_longueur_pond_hw) #longueur totale de l'échantillon avec majoration pondérée

14028145.5


In [106]:
total_longueur_pond_nv = df_a_velo['longueur_pond_adg'].sum()
print(total_longueur_pond_nv) #longueur totale de l'échantillon avec majoration pondérée

14382204.5


In [107]:
total_longueur = df_a_velo['longueur'].sum()
print(total_longueur)

14174798


In [108]:
total_longueur_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur'].sum().reset_index()

In [109]:
total_longueur_pond_hw_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond_hw'].sum().reset_index()

In [110]:
total_longueur_pond_adg_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond_adg'].sum().reset_index()

In [111]:
df_amenagements = pd.merge(total_longueur_commune, total_longueur_pond_hw_commune, on=['nom_com', 'insee_com'], suffixes=('_non_pond', '_pond_hw'))

In [112]:
df_amenagements = pd.merge(df_amenagements, total_longueur_pond_adg_commune, on=['nom_com', 'insee_com'], suffixes=(None, '_pond_adg'))

In [113]:
df_amenagements.sample(10)

,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg
267,Dammarie-les-Lys,77152,30862,31861.25,33337.50
929,Villaines-sous-Bois,95660,3260,3084.00,3305.75
86,Beynes,78062,27886,27517.00,27924.50
138,Brunoy,91114,30180,29856.00,31480.50
602,Montlignon,95426,10993,11361.25,12168.50
713,Pringy,77378,785,904.00,942.25
784,Saint-Lambert,78561,6022,6116.50,6197.50
94,Boinvilliers,78072,1855,1909.75,1855.00
889,Ussy-sur-Marne,77478,146,182.50,182.50
588,Montcourt-Fromonville,77302,4734,5337.00,4711.00


In [114]:
df_amenagements['difference_hw'] = df_amenagements['longueur_pond_hw'] - df_amenagements['longueur']
df_amenagements['difference_adg'] = df_amenagements['longueur_pond_adg'] - df_amenagements['longueur']

Nous avons créé nos trois indicateurs par commune : longueur et longueur pondérée (adg et hw) par commune, mais nous avons perdu de l'information dans cette opération puisque nous perdons de ce fait les coordonnées exactes des aménagements. Cependant, pour l'étude de la densité, cela pourra aussi s'avérer utile. Surtout nous avons besoin de ces indicateurs pour le travail économétrique que nous souhaitons mener, où nous considérons alors les communes comme des individus, et la longueur des pistes cyclables comme nous variable d'intérêt. Cependant, avant de passer à la modélisation économétrique pour tenter d'expliquer le développement des aménagements dans les différentes communes, nous souhaitons représenter et analyser spatialement la répartition des aménagements, leur densité (par habitant), leur densité (par kilomètre) et notamment compléter ces analyses grâce à la disponibilité des données du Vélib.

## Les vélibs en Ile de France

In [115]:
url2 = "https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

In [116]:
velib = gpd.read_file(url2)

In [123]:
velib.sample(n=15)

,capacity,name,stationcode,geometry
1093,50,Dom-Pérignon - Gravelle,12119,POINT (2.40988 48.82548)
28,43,Boulets - Faubourg Saint-Antoine,11010,POINT (2.39175 48.84926)
1123,34,Sommerard - Saint-Jacques,5002,POINT (2.34502 48.85031)
38,33,Saint-Fargeau - Mortier,20117,POINT (2.40791 48.87267)
857,30,Quai de l'Oise - Aisne,19128,POINT (2.38337 48.89062)
735,30,Gare de Robinson,23303,POINT (2.28183 48.77993)
990,33,Vistule - Choisy,13113,POINT (2.36157 48.82366)
98,30,Gare RER de Gentilly,42504,POINT (2.34103 48.81428)
815,33,Rossini - Laffitte,9022,POINT (2.33798 48.87335)
156,17,Galilée - Vernet,8003,POINT (2.29854 48.87172)


In [126]:
# Initialiser le géocodeur Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Fonction pour obtenir le code postal à partir des coordonnées
def get_postal_code(point):
    location = geolocator.reverse((point.y, point.x), language='fr')
    if location and location.raw.get('address', {}).get('postcode'):
        return location.raw['address']['postcode']
    else:
        return None

In [ ]:
velib['code_postal'] = velib['geometry'].apply(get_postal_code)

## Données et ressources nécessaires pour l'étude spatiale

In [ ]:
communes = download_vectorfile_url_all(
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    values = ["75","77","78","91", "92", "93", "94","95"],
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [ ]:
communes['surface'] = communes.to_crs(2154).area

In [ ]:
communes.sort_values('surface', ascending = False)

In [ ]:
communes.rename(columns={'INSEE_COG': 'insee_com'}, inplace=True)

## Données socio-démographiques sur l'Ile de France

## Données complémentaires : accidents de la route, transports écologiques alternatifs (réseau ferré, bus)

## Base finale avec la longueur, les indicateurs géographiques, et les variables socio-démographiques

In [ ]:
base = communes[['insee_com', 'POPULATION', 'surface', 'geometry']].merge(y, how='outer', on='insee_com')

In [ ]:
base.groupby('insee_com').sum(numeric_only = True).sort_values('longueur', ascending = False)

In [ ]:
base = gpd.GeoDataFrame(base, geometry='geometry')

# Analyse descriptive et spatiale

## Répartition spatiale des aménagements cyclables : piste cyclable et station vélib

In [ ]:
# Répartition des stations vélib

fig,ax = plt.subplots(figsize=(10, 10))
velib.plot(ax = ax, color = 'green')
com.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Emplacement des pistes cycables (point)

fig,ax = plt.subplots(figsize=(10, 10))
a_velo.plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
communes.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Répartition des pistes cyclabes en île de France

fig, ax = plt.subplots(figsize=(10, 10))
dissolved = base.dissolve(by='insee_com', aggfunc='sum').reset_index()
dissolved.plot(ax=ax, column="longueur", legend=True)
ax.set_axis_off()
legend = ax.get_legend()
plt.show()

### Statistiques descriptives sur les pistes cyclables

In [ ]:
df_amenagements.describe()

In [ ]:
sns.scatterplot(df_amenagements['longueur'])
plt.title('Distribution des pistes cyclables dans les communes')